In [79]:
import pandas
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import MySQLdb as mdb

In [80]:
con = mdb.connect('localhost', 'root', '', 'climbfinder_test')

In [81]:
def get_refclimb(name, crag = None):
    if crag == None:
        sql  = "SELECT * FROM Features WHERE name LIKE '%(name)s'" %{"name": name}
    else:
        sql  = "SELECT * FROM Features WHERE name LIKE '%(name)s' AND crag LIKE '%(crag)s'" %{"name": name, "crag": crag}
    
    ref_climb = pandas.read_sql_query(sql=sql, con=con) 
    
    return ref_climb

In [82]:
def get_comparisonclimbs(region = None, yes = 'yes', no = 'no'):
    if region != None:
        sql  = "SELECT * FROM Features WHERE region LIKE '%(region)s' AND %(yes)s = 1 AND %(no)s = 0" %{"region": region,
                                                                                                      "yes": yes, 
                                                                                                      "no":no}
    else: 
        sql  = "SELECT * FROM Features WHERE %(yes)s = 1 AND %(no)s = 0" %{"yes": yes, "no":no}
    
    comparisonclimbs = pandas.read_sql_query(sql=sql, con=con)
     
    return comparisonclimbs

In [166]:
def get_max_sim(ref_climb, comparisonclimbs, n = 5,):
    features = ['toprope',
       'sport', 'trad', 'alpine', 'ice', 'aid', 'bouldering', 'pitches',
       'grade_num', 'anchor', 'arete', 'awkward', 'block',
       'bolt', 'boulder', 'bulge', 'broken', 'canyon', 'buttress',
       'chains', 'chimney', 'classic', 'clean','corner', 'crack',
       'crimps', 'crux', 'difficult', 'dihedral', 'exposure', 'face',
       'finger', 'flake', 'gully', 'hand', 'jug', 'ledge', 'lip',
       'jam', 'notch', 'moderate', 'ridge', 'vertical', 'horizontal',
       'mantle', 'offwidth', 'overhang','pillar', 'pocket', 'protect',
       'ramp', 'rappel', 'roof', 'scramble', 'seam', 'slab', 'sloper',
       'solid', 'stance', 'steep', 'stemming', 'sustained', 'technical',
       'tower', 'traverse', 'undercling', 'average_rating']

           
    comparisonclimbs_features = comparisonclimbs.ix[:,features]
    ref_climb_features = ref_climb[features]
    
    if len(comparisonclimbs) == 0:
        topfive = None
        return topfive
    
    similarities = cosine_similarity(ref_climb_features, comparisonclimbs_features)
    similarities_df = pandas.DataFrame(similarities).transpose().sort(0, ascending = False)
    
    
    if similarities_df.index[0] == ref_climb['index'][0]:
        similarities_df = similarities_df[1:]
        
    if len(similarities_df) >= 5:
        topfive = similarities_df[:5]
    elif len(similarities_df) <= 5:
        if len(similarities_df) > 0:
            topfive = similarities_df[:len(similarities_df)]
        else: 
            topfive = None
    
    return similarities_df

In [107]:
def get_climb_info(topfive, comparisonclimbs):
    climb_info = comparisonclimbs.ix[topfive.index,:]
    return climb_info

In [148]:
ref

,index,name,area,climb_info,crag,region,description,number_of_votes,grade,protection,...,stance,steep,stemming,sustained,technical,tower,traverse,undercling,yes,no
0,9,son of easy o,frog's head - the arch,"Trad, 2 pitches, 170'",the gunks,new york,This may be the best 5.8 at the Gunks if you l...,262,5.8,Standard rack; extra finger-size pro for combi...,...,0,1,0,0,0,0,1,0,1,0


In [132]:
ref = get_refclimb('son of easy o')
ref

,index,name,area,climb_info,crag,region,description,number_of_votes,grade,protection,...,stance,steep,stemming,sustained,technical,tower,traverse,undercling,yes,no
0,9,son of easy o,frog's head - the arch,"Trad, 2 pitches, 170'",the gunks,new york,This may be the best 5.8 at the Gunks if you l...,262,5.8,Standard rack; extra finger-size pro for combi...,...,0,1,0,0,0,0,1,0,1,0


In [133]:
comp = get_comparisonclimbs()

In [135]:
info = get_climb_info(top, comp)

In [167]:
topfive = get_max_sim(ref,comp)

0
0


In [168]:
topfive

,0
0,0.916984
816,0.906671
40195,0.904943
21237,0.901339
3079,0.897801
26614,0.897510
18960,0.897510
17622,0.897495
27660,0.895173
50674,0.895173
